In [2]:
import os
import sys
import pandas as pd
import numpy as np
import glob
import time
from scipy.sparse import csr_matrix
import anndata as an
import scanpy as sc
import pyranges as pr
import psutil

In [3]:
fpath = "/scratch/indikar_root/indikar1/shared_data/higher_order/anndata/population_mESC_1000000_raw.h5ad"

start_time = time.time()  # Record the start time
adata = sc.read_h5ad(fpath)
end_time = time.time()  # Record the end time
print(f"Time taken to read the file: {end_time - start_time:.2f} seconds")
sc.logging.print_memory_usage()
adata

Time taken to read the file: 60.55 seconds
Memory usage: current 2.99 GB, difference +2.99 GB


AnnData object with n_obs × n_vars = 2579 × 2756467
    obs: 'bin_index', 'bin_start', 'bin_end', 'bin', 'chrom', 'chrom_bin', 'degree', 'genes', 'n_genes'
    var: 'read_index', 'basename', 'mean_mapq', 'median_mapq', 'n_chromosomes', 'order', 'n_bins', 'read_length_bp', 'genes', 'n_genes'
    uns: 'base_resolution', 'chrom_sizes', 'gdf', 'gene_map', 'intervals'
    layers: 'H'

In [4]:
outpath = "/scratch/indikar_root/indikar1/shared_data/higher_order/gene_maps/"
gene_map = adata.uns['gene_map']
gene_map.head()

,gene_name,gene_biotype,read_name,bin_name
0,Ulk4,protein_coding,3891ee6d-53d1-4ee0-ba2f-3d22291d4493,chr9:121
1,Smarca2,protein_coding,66953ddf-e76d-4cdf-aaf8-be028a2d7b04,chr19:26
2,Dlgap3,protein_coding,ad5b2240-893f-4ed0-a157-c2be66d8d754,chr4:127
3,Gm56531,lncRNA,3f354c45-5e48-4f6d-8c7e-05369432b344,chr12:8
4,Twsg1,protein_coding,d4626feb-16a2-4aac-8145-53e89b60bf7c,chr17:66


In [ ]:
break

In [ ]:
pore_c_path = "/scratch/indikar_root/indikar1/shared_data/higher_order/pore_c/population_mESC.read_level.parquet"
resolution = 1000000
chrom_path = "/scratch/indikar_root/indikar1/shared_data/higher_order/reference/chrom_sizes.csv"
gene_path = "/scratch/indikar_root/indikar1/shared_data/higher_order/reference/gene_table.parquet"

In [ ]:
# Load the Pore-C data
mk.print_section_header("Loading Pore-C Data")
df = pd.read_parquet(pore_c_path)
df['value'] = 1
mk.print_data_shape("Pore-C data", df.shape)
mk.print_memory_usage("Load Pore-C data")

n = df.shape[0]

In [ ]:
# Load the chromosome table
mk.print_section_header("Creating Chromosome Intervals")
chrom, intervals = mk.create_chromosome_intervals(chrom_path, base_resolution=resolution)
mk.print_data_shape("Chromosome intervals", intervals.shape)

In [ ]:
intervals[intervals['chrom'] == '2'].head()

In [ ]:
df = mk.merge_genes(df, gene_path)
print(f"{df.shape=}")
df.head()

In [ ]:
n - df.shape[0]

In [ ]:
# Add the interval information
mk.print_section_header("Joining Intervals with Pore-C Data")
df = mk.join_intervals_pyranges(df, intervals)
df.head()

In [ ]:
n - df.shape[0]

In [ ]:
def create_var_df(df, var_names):
  """Creates a variable DataFrame from a DataFrame with read information.

  Args:
    df (pandas.DataFrame): DataFrame with read data including 'read_name', 'read_index', 
                           'mapping_quality', 'chrom', 'order', 'bin', and 'length_on_read'.
    var_names (pandas.Index): Index of unique read names.

  Returns:
    pandas.DataFrame: DataFrame containing variable information (read-level summaries).
  """
  var = df.copy()

  gene_list = lambda x: ";".join([i for i in set(x) if i != '-1'])
  n_genes = lambda x: len([i for i in set(x) if i != '-1'])

  var = var.groupby(['basename', 'read_name', 'read_index']).agg(
      mean_mapq=('mapping_quality', 'mean'),
      median_mapq=('mapping_quality', 'median'),
      n_chromosomes=('chrom', 'nunique'),
      order=('order', 'first'),
      n_bins=('bin', 'nunique'),
      read_length_bp=('length_on_read', 'sum'),
      genes=('gene_name', gene_list),
      n_genes=('gene_name', n_genes),
  ).reset_index()

  # Ensure proper sorting using var_names
  var = var.set_index('read_index')
  # var = var.reindex(var_names)
  # var = var.reset_index()
  # var = var.set_index('read_name')

  return var


var = create_var_df(df, "")
var.head()

In [ ]:
def create_obs_df(df, obs_names):
  """Creates an observation DataFrame from a DataFrame and a sparse matrix.

  Args:
    df (pandas.DataFrame): DataFrame with genomic data and bin information.
    X (scipy.sparse.csr_matrix): Sparse matrix representation of the data.

  Returns:
    pandas.DataFrame: DataFrame containing observation information.
  """
  gene_list = lambda x: ";".join([i for i in set(x) if i != '-1'])
  n_genes = lambda x: len([i for i in set(x) if i != '-1'])
    
  obs = df.groupby('bin_name').agg(
      bin_start=('bin_start', 'first'),
      bin_end=('bin_end', 'first'),
      bin=('bin', 'first'),
      bin_index=('bin_index', 'first'),
      chrom_bin=('chrom_bin', 'first'),
      degree=('read_name', 'nunique'),
      genes=('gene_name', gene_list),
      n_genes=('gene_name', n_genes),
  ).reset_index()


  # Ensure proper sorting using var_names
  obs = obs.set_index('bin_index')
  obs = obs.reindex(obs_names)
  obs = obs.reset_index()
  obs = obs.set_index('bin_name')

  return obs

obs = create_obs_df(df, "")
obs.head()

In [ ]:
break

In [ ]:
df.head()

In [ ]:
gene_map = df[['gene_name', 'gene_biotype', 'read_name', 'bin_name']].drop_duplicates()
gene_map = gene_map[gene_map['gene_name'] != '-1']
gene_map = gene_map.reset_index(drop=True)

gene_map.head()

In [ ]:
break

In [ ]:
df.columns

In [ ]:
df['duplicate_maps'] = df.groupby(['read_name', 'read_start'])['gene_name'].transform('nunique')
test = df[df['duplicate_maps'] > 1]
test = test.sort_values(by=['read_name', 'ref_start'])
print(f"{test.shape=}")
test[['read_name', 'read_start', 'ref_start', 'chrom', 'gene_name', 'gene_biotype', 'gene_start']].head()

In [ ]:
break

In [ ]:
df['gene_name'].value_counts()

In [ ]:
df['read_name'].value_counts()

In [ ]:
gdf = pd.read_parquet(gene_path)
print(f"{gdf.shape=}")
gdf.head()

# gdf_pr = pr.PyRanges(gdf)


In [ ]:
df.head()